In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from os import listdir, walk
from os.path import isfile, join

In [2]:
%run load_dataset.ipynb

Loading 1001's data
     sid   raw_label                timestamp  duration   label
0   1001       stand  2019-01-14 14:52:30.956  00:00:30   stand
1   1001         sit  2019-01-14 14:53:07.802  00:00:30     sit
2   1001       stand  2019-01-14 14:53:41.932  00:00:28   stand
3   1001       sleep  2019-01-14 14:55:15.521  00:00:30   sleep
4   1001         sit  2019-01-14 14:55:51.726  00:00:30     sit
5   1001       sleep  2019-01-14 14:56:27.937  00:00:31   sleep
6   1001         sit  2019-01-14 14:57:03.592  00:00:01     sit
7   1001       stand  2019-01-14 14:57:05.548  00:00:31   stand
8   1001        walk  2019-01-14 14:59:11.365  00:01:00    walk
9   1001  downstairs  2019-01-14 15:01:41.375  00:00:13  stairs
10  1001    upstairs  2019-01-14 15:01:56.600  00:00:21  stairs
11  1001  downstairs  2019-01-14 15:02:20.945  00:00:22  stairs
12  1001    upstairs  2019-01-14 15:02:44.824  00:00:22  stairs
Loading 1002's data
     sid   raw_label                timestamp  duration   label


Loading 2002's data
     sid   raw_label                timestamp  duration   label
0   2002       stand  2019-01-16 14:22:12.439  00:00:30   stand
1   2002         sit  2019-01-16 14:22:46.157  00:00:30     sit
2   2002       stand  2019-01-16 14:23:19.307  00:00:30   stand
3   2002       sleep  2019-01-16 14:24:05.584  00:00:30   sleep
4   2002         sit  2019-01-16 14:24:44.454  00:00:30     sit
5   2002       sleep  2019-01-16 14:25:20.640  00:00:30   sleep
6   2002       stand  2019-01-16 14:25:59.460  00:00:30   stand
7   2002        walk  2019-01-16 14:26:42.431  00:00:30    walk
8   2002        walk  2019-01-16 14:27:17.169  00:00:31    walk
9   2002  downstairs  2019-01-16 14:28:22.139  00:00:23  stairs
10  2002    upstairs  2019-01-16 14:28:51.399  00:00:20  stairs
11  2002  downstairs  2019-01-16 14:29:15.654  00:00:21  stairs
12  2002    upstairs  2019-01-16 14:29:39.313  00:00:20  stairs
Finished loading


In [3]:
%run preprocess_for_SVM.ipynb

In [4]:
subj_train = ['1001','1002','1003','2001']
subj_val = ['1004','1005','1006','2002']
subj_test = ['1007','1008','1009']

In [5]:
def get_X_y(subj_ids, X, y, s):
    X_get = []
    y_get = []
    s_get = []
    for i in range(len(s)):
        for j in range(len(subj_ids)):
            if(s[i]==subj_ids[j]):
                X_get.append(X[i])
                y_get.append(y[i])
                s_get.append(s[i])
                
    return np.array(X_get), np.array(y_get), np.array(s_get)

In [6]:
X_train, y_train, s_train = get_X_y(subj_train, X_all, y_all, subj_all)
X_val, y_val, s_val = get_X_y(subj_val, X_all, y_all, subj_all)
X_test, y_test, s_test = get_X_y(subj_test, X_all, y_all, subj_all)

In [8]:
print(X_train.shape, y_train.shape, s_train.shape)
print(X_val.shape, y_val.shape, s_val.shape)
print(X_test.shape, y_test.shape, s_test.shape)

(8311, 3) (8311,) (8311,)
(7714, 3) (7714,) (7714,)
(5982, 3) (5982,) (5982,)


In [7]:
print(X_train.shape, y_train.shape)

(8311, 3) (8311,)


In [10]:
def prepare_pure(X, y, subj_all, new_label_list):
    X_label, y_label = label_grouping(X, y, subj_all, new_label_list)
    
    X_concat = []
    y_concat = []
    for i in range(len(X_label)):
        for j in range(len(X_label[i])):
            X_ol, y_ol = make_overlapping(np.array(X_label[i][j]), y_label[i][j])
            print(i, j, X_ol.shape)
            
            if(len(X_concat)==0):
                X_concat = X_ol
            else:
                X_concat = np.vstack((X_concat, X_ol))

            if(len(y_concat)==0):
                y_concat = y_ol
            else:
                y_concat = np.hstack((y_concat, y_ol))
    
    return X_concat, y_concat    

In [9]:
X_label, y_label = label_grouping(X_train, y_train, s_train, label_list)
print(len(X_label[4][2]))

0


In [11]:
label_list = [0,1,3,2,4]
label_dict = {
    0: 'sit',
    1: 'sleep',
    3: 'stand',
    2: 'stairs',
    4: 'walk'
}

all_subjects = subj_train
X_train_pure, y_train_pure = prepare_pure(X_train, y_train, s_train, label_list)

all_subjects = subj_val
X_val_pure, y_val_pure = prepare_pure(X_val, y_val, s_val, label_list)

all_subjects = subj_test
X_test_pure, y_test_pure = prepare_pure(X_test, y_test, s_test, label_list)

0 0 (322, 60, 3)
0 1 (314, 60, 3)
0 2 (326, 60, 3)
0 3 (315, 60, 3)
1 0 (320, 60, 3)
1 1 (314, 60, 3)
1 2 (314, 60, 3)
1 3 (314, 60, 3)
2 0 (494, 60, 3)
2 1 (507, 60, 3)
2 2 (501, 60, 3)
2 3 (501, 60, 3)
3 0 (289, 60, 3)
3 1 (228, 60, 3)
3 2 (190, 60, 3)
3 3 (617, 60, 3)
4 0 (315, 60, 3)
4 1 (315, 60, 3)
4 2 (315, 60, 3)
4 3 (320, 60, 3)
0 0 (321, 60, 3)
0 1 (302, 60, 3)
0 2 (327, 60, 3)
0 3 (315, 60, 3)
1 0 (127, 60, 3)
1 1 (327, 60, 3)
1 2 (332, 60, 3)
1 3 (315, 60, 3)
2 0 (458, 60, 3)
2 1 (518, 60, 3)
2 2 (507, 60, 3)
2 3 (500, 60, 3)
3 0 (216, 60, 3)
3 1 (158, 60, 3)
3 2 (209, 60, 3)
3 3 (339, 60, 3)
4 0 (314, 60, 3)
4 1 (314, 60, 3)
4 2 (315, 60, 3)
4 3 (320, 60, 3)
0 0 (334, 60, 3)
0 1 (327, 60, 3)
0 2 (322, 60, 3)
1 0 (314, 60, 3)
1 1 (321, 60, 3)
1 2 (315, 60, 3)
2 0 (508, 60, 3)
2 1 (439, 60, 3)
2 2 (502, 60, 3)
3 0 (197, 60, 3)
3 1 (247, 60, 3)
3 2 (296, 60, 3)
4 0 (315, 60, 3)
4 1 (314, 60, 3)
4 2 (346, 60, 3)


In [12]:
print(X_train_pure.shape, y_train_pure.shape)
print(X_val_pure.shape, y_val_pure.shape)
print(X_test_pure.shape, y_test_pure.shape)

(7131, 60, 3) (7131,)
(6534, 60, 3) (6534,)
(5097, 60, 3) (5097,)


In [13]:
VAL_LEN = X_val.shape[0]
X_train = X_train[:VAL_LEN]
y_train = y_train[:VAL_LEN]

In [14]:
print("Hello World!")

Hello World!


In [15]:
import keras

Using TensorFlow backend.


In [17]:
%run RNN_model.ipynb